In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import gc
import random

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_dir = '../input/train'
test_dir = '../input/test'
train_dogs = ['../input/train/{}'.format(i) for i in os.listdir(train_dir) if 'dog' in i]
train_cats = ['../input/train/{}'.format(i) for i in os.listdir(train_dir) if 'cat' in i]
test_images = ['../input/test/{}'.format(i) for i in os.listdir(test_dir)]
batch = train_dogs[0:3000]+train_cats[0:3000]
random.shuffle(batch)












In [ ]:
import matplotlib.image as mpimg
for ima in batch[0:10]:
    img = mpimg.imread(ima)
    imgplot = plt.imshow(img)
    plt.show()

In [ ]:
def process(image_list):
    X = []
    y = []
    for image in image_list:
        X.append(cv2.resize(cv2.imread(image,cv2.IMREAD_COLOR),(150,150),interpolation=cv2.INTER_CUBIC))
        if('dog' in image):
            y.append(1)
        else:y.append(0)
    return X,y


In [ ]:
X,y = process(batch)
print(len(X))

In [ ]:
print(X[0])

In [ ]:
X = np.array(X)
y = np.array(y)
X[0].shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2)


In [ ]:
batch_size = 32
n_train = len(X_train)
n_test = len(X_val)

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array,load_img

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation = 'relu',input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation = 'relu'))
model.add(layers.Dense(1,activation = 'relu'))
model.summary()













In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=0.0001), metrics = ['accuracy'])


In [ ]:
train_data = ImageDataGenerator(rescale=1./255)
train_generator = train_data.flow(X_train,y_train,batch_size=batch_size)
test_generator = train_data.flow(X_val,y_val,batch_size=batch_size)


In [ ]:
history=model.fit_generator(train_generator,epochs=20,steps_per_epoch=100)

In [ ]:
X_test,y_test = process(test_images[0:15])
x = np.array(X_test)
test_datagen = ImageDataGenerator(rescale=1./255)









In [ ]:
i=0
labels=[]
plt.figure(figsize=(30,20))
for batch in test_datagen.flow(x,batch_size=1):
    pred = model.predict(batch)
    if(pred>=0.5):labels.append('dog')
    else:labels.append('cat')
    
    plt.title(labels[i])
    imgplot=plt.imshow(batch[0])
    i+=1
    if(i==15):break
    plt.show()